# Explore here

In [38]:
import pandas as pd
import numpy as np
import re
nltk.download('stopwords')
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
import joblib
import os

[nltk_data] Downloading package stopwords to /home/vscode/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Paso 1: Carga del conjunto de datos

In [39]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv"
df = pd.read_csv(url)

In [40]:
df.head()

,url,is_spam
0,https://briefingday.us8.list-manage.com/unsubs...,True
1,https://www.hvper.com/,True
2,https://briefingday.com/m/v4n3i4f3,True
3,https://briefingday.com/n/20200618/m#commentform,False
4,https://briefingday.com/fan,True


In [41]:
df.shape

(2999, 2)

## Paso 2.1: Limpieza y segmentación de las URLs

In [42]:
# Función de limpieza de URLs
def clean_url(url):
    # Convertir a minúsculas
    url = url.lower()
    
    # Reemplazar símbolos comunes por espacios
    url = re.sub(r'[\/\.\-\?\=\_]', ' ', url)

    # Eliminar cualquier cosa que no sea alfanumérica
    url = re.sub(r'[^a-z0-9 ]', '', url)

    # Quitar múltiples espacios
    url = re.sub(r'\s+', ' ', url).strip()

    return url

In [43]:
# Stopwords personalizadas
custom_stopwords = set(stopwords.words('english'))  # del módulo NLTK
custom_stopwords.update(['http', 'https', 'www', 'com', 'net', 'html', 'php', 'index', 'org', 'onion', 'to']) # palabras comunes en URLs

In [44]:
# Aplicar limpieza
df['clean_url'] = df['url'].apply(clean_url)

In [45]:
# Ver los valores unicos de la variable 'is_spam'
df['is_spam'].unique()

array([ True, False])

In [46]:
# Convertir variable 'is_spam' en numerica
df['label'] = df['is_spam'].astype(int)

In [47]:
df.head()

,url,is_spam,clean_url,label
0,https://briefingday.us8.list-manage.com/unsubs...,True,https briefingday us8 list manage com unsubscribe,1
1,https://www.hvper.com/,True,https www hvper com,1
2,https://briefingday.com/m/v4n3i4f3,True,https briefingday com m v4n3i4f3,1
3,https://briefingday.com/n/20200618/m#commentform,False,https briefingday com n 20200618 mcommentform,0
4,https://briefingday.com/fan,True,https briefingday com fan,1


In [48]:
# Paso 2.2: División Train/Test
X = df['clean_url']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [49]:
# Paso 2.3: Vectorización (TF-IDF)
vectorizer = TfidfVectorizer(stop_words=list(custom_stopwords))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

## Paso 3: Implementar SVM con parámetros por defecto

In [50]:
# Crear el modelo SVM con parámetros por defecto
svm = SVC()

# Entrenar con los datos vectorizados de entrenamiento
svm.fit(X_train_vec, y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [51]:
# Predecir en el conjunto de test
y_pred = svm.predict(X_test_vec)

In [52]:
# Evaluar resultados
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9516666666666667

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.96      0.98      0.97       461
           1       0.93      0.86      0.89       139

    accuracy                           0.95       600
   macro avg       0.94      0.92      0.93       600
weighted avg       0.95      0.95      0.95       600


Matriz de confusión:
 [[452   9]
 [ 20 119]]


### Hallazgos
- **Alta precisión general (95%):** 
El modelo clasifica correctamente el 95% de las URLs del conjunto de prueba.

- **Detecta muy bien las URLs no spam:** 
La clase "no spam" (0) tiene una precisión del 96% y recall del 98%.

- **Buen desempeño con spam, pero podría mejorar el recall:** 
Aunque identifica correctamente el 93% de las predicciones como spam, su recall es de 86%.

## Paso 4: Optimizar el modelo anterior

In [53]:
# Definir el espacio de búsqueda
param_grid = {
    'C': [0.1, 1, 10], #  # C controla la penalización del error
    'kernel': ['linear', 'rbf', 'poly'], # # gamma controla la influencia de cada punto
    'gamma': ['scale', 'auto'] 
}

# Modelo base
svm = SVC()

# Grid Search con validación cruzada de 5 folds
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='f1', n_jobs=-1) # Métrica objetivo: f1-score (porque queremos mejorar recall sin perder precisión)

# Entrenamiento con datos vectorizados
grid_search.fit(X_train_vec, y_train)

# Resultados
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor score (F1):", grid_search.best_score_)

Mejores hiperparámetros: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Mejor score (F1): 0.926222222996787


In [54]:
# Predecir con el mejor modelo
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_vec)

# Evaluar
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_test, y_pred))
print("\nMatriz de confusión:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9466666666666667

Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.98      0.95      0.96       461
           1       0.86      0.92      0.89       139

    accuracy                           0.95       600
   macro avg       0.92      0.94      0.93       600
weighted avg       0.95      0.95      0.95       600


Matriz de confusión:
 [[440  21]
 [ 11 128]]


### Hallazgos

- Se mantuvo una alta precisión general (accuracy: 94.7%).
- Mejora notable en la detección de spam (clase 1): antes, el modelo tenía un recall de 0.86 para spam y ahora es 0.92.
- Buen equilibrio entre clases. El F1-score para spam (clase 1) es de 0.89, lo que indica un buen equilibrio entre precisión (0.86) y recall (0.92). Además, la clase no-spam (clase 0) mantiene un rendimiento muy alto (f1-score de 0.96).

**Conclusión:**

Gracias a la optimización, el modelo sigue clasificando bien los correos legítimos, mejora la detección de spam, y reduce errores críticos como dejar pasar correos spam como si fueran buenos.

## Paso 5: Guardar el modelo

In [55]:
# Ruta donde guardar el modelo
model_dir = '/workspaces/samiamarante-NLP/models'
os.makedirs(model_dir, exist_ok=True)

# Guarda el modelo y el vectorizador
joblib.dump(grid_search.best_estimator_, os.path.join(model_dir, 'svm_spam_model.pkl'))
joblib.dump(vectorizer, os.path.join(model_dir, 'tfidf_vectorizer.pkl'))

['/workspaces/samiamarante-NLP/models/tfidf_vectorizer.pkl']